# SQL Server

## Introduction

In this notebook, we will demonstrate the use of the SQL server workflow.
This workflow runs a server that provides access to an ApertureDB instance via queries in the SQL language, specifically the PostgreSQL dialect.

For more information, see [the documentation for this workflow](https://docs.aperturedata.io/workflows/sql_server).

## Setup

In order to run this notebook, you will need to be running the SQL server workflow.
You can do this conveniently in the Cloud UI.

You will also need to have some data stored in your ApertureDB, perhaps from running the "Website Chatbot" or "Dataset Ingestion" workflows.

You will also need to know the password for the SQL server.
You can find that in the "Connection Helper" dialog in the Cloud UI.

In [ ]:
import psycopg
from getpass import getpass
import os
import json
import pandas as pd
from aperturedb import NotebookHelpers as nh

## Enter Password

In [ ]:
password = getpass("password")

## Set up the client connection

We're using the `psycopg` PostgreSQL database adaptor. 

In [ ]:
# If you are not runing this notebook in the ApertureDB Cloud, then you may need to set the host below.
host = "<DB_HOST>"
database = "aperturedb"
user = "aperturedb"

def run_query(query):
    """Run a query and return the results."""
    with psycopg.connect(
            f"dbname={database} user={user} password={password} host={host}") as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            return pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

## Test the connection

If everything is set up correctly, this will print something like:
```
   ping
0	1
```

In [ ]:
run_query("SELECT 1 AS ping")

## List tables

Let's find out what tables the SQL server has.

Our tables are divided up into four schemata:
* `system`: This schema contains a table for every system object type, including one for `Entity` and one for `Connection`. These correspond to the `Find...` commands in the ApertureDB Query Language.
* `entity`: This schema contains a table for every user-defined entity class. These correspond to the values you can use with `with_class` in `FindEntity` commands.
* `connection`: This schema contains a table for every user-defined connection class. These correspond to the values you can use with `with_class` in `FindConnection` commands or the `connection_class` field in the `is_connected_to` parameter.
* `descriptor`: This schema contains a table for every descriptor set, effectively the values you can use with the `set` parameter in `FindDescriptor`.

Note that it isn't necessary to give the schema explicitly when referring to a table, unless the table name is ambiguous, because all four schemata are on the search path.
You will generally have to enclose table names in double quotes because their names are mixed case and can contain special characters, whereas unquoted identifiers in SQL are somewhat restricted.

In [ ]:
def list_tables(schema):
    return run_query(f"""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = {schema!r}
        ORDER BY table_name;
        """)

for schema in ['system', 'entity', 'connection', 'descriptor']:
    tables = list_tables(schema)
    print(f"Tables in schema '{schema}' ({len(tables)} total):")
    display(tables)


## Find some entities

Two ways to find entities. We can use the `system."Entity"` table to look at all entities, or a `entity.…` table to look at a specific entity type.

In [ ]:
display(run_query("SELECT * FROM system.\"Entity\" LIMIT 5;"))

entity_class = list_tables('entity').table_name[0]
print(f"Using entity class: {entity_class}")
display(run_query(f"SELECT * FROM entity.\"{entity_class}\" LIMIT 5;"))

## Find some connections

We can do the same two types of query to find connections.
In the language of relational databases, you might like to think of ApertureDB connections as "join tables" or ["associative entities](https://en.wikipedia.org/wiki/Associative_entity)".

In [ ]:
display(run_query("SELECT * FROM system.\"Connection\" LIMIT 5;"))

connection_class = list_tables('connection').table_name[0]
print(f"Using connection class: {connection_class}")
display(run_query(f"SELECT * FROM connection.\"{connection_class}\" LIMIT 5;"))

## Put it together: Graph query

One of the features of ApertureDB is that it acts as a graph database.
You can see this in SQL by joining tables together.


In [ ]:
display(run_query("""
SELECT A.*, B.*, C.* FROM entity.\"CrawlRun\" AS A
INNER JOIN connection.\"crawlRunHasDocument\" AS B ON A._uniqueid = B._src
INNER JOIN entity.\"CrawlDocument\" AS C ON B._dst = C._uniqueid
LIMIT 5;
"""))

## Look under the covers

So we know that these SQL queries are somehow being turned into ApertureDB Query Language queries under the covers.
Is there any way to see the actual AQL queries?
Yes, by using the EXPLAIN feature of SQL.

You should be able to find two `FindEntity`s and a `FindConnection`.

In [ ]:
for row in run_query("""
    EXPLAIN
    SELECT A.*, B.*, C.* FROM entity.\"CrawlRun\" AS A
    INNER JOIN connection.\"crawlRunHasDocument\" AS B ON A._uniqueid = B._src
    INNER JOIN entity.\"CrawlDocument\" AS C ON B._dst = C._uniqueid
    LIMIT 5;
    """)['QUERY PLAN'].tolist():
    print(row)

## Find some images

In addition to being a graph database, ApertureDB is also a multi-modal object store.
This means that objects can have associated blobs, and ApertureDB can perform various operations on, say, images.

For objects that have an associated blob, such as images, we can fetch those blobs as part of the SQL query.
The blobs end up in a special field, here `_image` of type `BYTEA`.

It can be expensive to fetch blob data, and so ApertureDB never does so by default, only when [the `blobs` parameter](https://docs.aperturedata.io/query_language/Reference/shared_command_parameters/blobs) is set.
To ensure that blobs are not returned casually when the user asks for `SELECT *`, a special column `_blobs` must be set to ask for blobs.

Another special column here is `_as_format`, which lets you pick the format of the image blobs.

In [ ]:
image_results = run_query("""
SELECT * FROM system."Image" 
WHERE _blobs
AND _as_format = 'jpg'
LIMIT 5
""")
display(image_results)
blobs = image_results['_image'].tolist()
nh.display(blobs)

## Image Operations

Another special column here is `_operations`. In combination with SQL functions we have defined, we can generate a pipeline of [operations](https://docs.aperturedata.io/query_language/Reference/shared_command_parameters/operations) that mutate the blob.

Finally, we can extract the actual blobs and display them. The transformations have made them pretty unrecognizable.

In [ ]:
image_results = run_query("""
SELECT * FROM system."Image" 
WHERE _blobs
AND _operations = OPERATIONS(
	THRESHOLD(64), 
	CROP(x:=10, y:=10, width:=200, height:=200),
	FLIP(+1),
	ROTATE(angle:=90),
	RESIZE(width:=50))
AND _as_format = 'jpg'
LIMIT 5
""")
display(image_results)
blobs = image_results['_image'].tolist()
nh.display(blobs)

## Under the covers

That was a complicated SQL query.
What did that get transformed into in ApertureDB?
It turn out that the transformation is pretty direct.

You'll see the `blobs`, `as_format`, and `operations` parameters appearing in the `FindImage` command body.

In [ ]:
for row in run_query("""
    EXPLAIN
    SELECT * FROM system."Image" 
    WHERE _blobs
    AND _operations = OPERATIONS(
        THRESHOLD(64), 
        CROP(x:=10, y:=10, width:=200, height:=200),
        FLIP(+1),
        ROTATE(angle:=90),
        RESIZE(width:=50))
    AND _as_format = 'jpg'
    LIMIT 5
    """)['QUERY PLAN'].tolist():
    print(row)

## Similarity search

In addition to being a graph database and an object store, ApertureDB is also a vector store.
We're going use this to find documents that are similar to an input text.

This example assumes that you have already run the "Website Chatbot" workflow.

In [ ]:
text = "find entity"  # query text
similarity_results = run_query(f"""
SELECT * FROM descriptor."crawl-to-rag"
WHERE _find_similar = FIND_SIMILAR(
    text := {text!r},
    k := 10)
LIMIT 10
""")
display(similarity_results)


## Under the covers

This query is slightly different because the wrapper is performing the embedding for us.

In [ ]:
for row in run_query(f"""
    EXPLAIN VERBOSE
    SELECT * FROM descriptor."crawl-to-rag"
    WHERE _find_similar = FIND_SIMILAR(
        text := {text!r},
        k := 10)
    LIMIT 10
    """)['QUERY PLAN'].tolist():
    print(row)

An addition here (in `VERBOSE` mode) is that we have generated a query blob as can be seen in the last line.
Recall that the underlying ApertureDB interface receives two inputs: A JSON query and an optional list of binary blobs.
The query blobs are commonly used by commands like `AddImage` to upload data into ApertureDB.
`FindDescriptor` is special among `Find` commands because it can also take a query blob, representing the embedding of the query text or image.

Here the SQL wrapper knows the embedding model used to create this descriptor set, and is able to generate an embedding for the text supplied.
It will also do the same for images using `FIND_SIMILAR(IMAGE:=...)`.
It can also accept a plain vector as a JSON string.

## Conclusion

Here we have seen that the SQL Server Workflow can provide read-only access to a broad range of ApertureDB features, including graph queries, object retrieval and manipulation, and vector search.